## Importando as bibliotecas

In [59]:
import pandas as pd
import numpy as np
import requests
import sys
import os

## Importando as funções do nba_scraper.py

In [60]:
from src.nba_scraper import find_ID_by_name
from src.nba_scraper import find_abb_by_name
from src.nba_scraper import getting_ID_row_by_name
from src.nba_scraper import get_four_factors
from src.nba_scraper import get_advanced_stats
from src.nba_scraper import get_scoring_stats
from src.nba_scraper import get_bench_point_percent
from src.nba_scraper import get_shot_locations_pct
from src.nba_scraper import get_usage_stars
from src.nba_scraper import get_all_teams_df

-------------------------------------------------------------------------------------------------------------------------------------------------------------

# Fazendo o Dataframe inicial dos ultimos 10 campeões
### Primeiramente colocarei, além da temporada, o 'full_name' do time, a temporada e o ID de cada time.

In [61]:
seasons = ["2015-16","2016-17","2017-18","2018-19","2019-20","2020-21",
           "2021-22","2022-23","2023-24","2024-25"]

teams_Series = ["CLEVELAND CAVALIERS", "GOLDEN STATE WARRIORS",
                      "GOLDEN STATE WARRIORS", "TORONTO RAPTORS","LOS ANGELES LAKERS",
                      "MILWAUKEE BUCKS", "GOLDEN STATE WARRIORS", "BOSTON CELTICS",
                        "DENVER NUGGETS", "OKLAHOMA CITY THUNDER"]

indexs = ["2016_CLE", "2017_GSW", "2018_GSW", "2019_TOR", "2020_LAL", "2021_MIL", "2022_GSW", "2023_BOS", "2024_DEN", "2025_OKC"]

df_last_10_champions = pd.DataFrame(index=indexs)
df_last_10_champions["TEAMS"] = teams_Series
df_last_10_champions["SEASON"] = seasons
df_last_10_champions

,TEAMS,SEASON
2016_CLE,CLEVELAND CAVALIERS,2015-16
2017_GSW,GOLDEN STATE WARRIORS,2016-17
2018_GSW,GOLDEN STATE WARRIORS,2017-18
2019_TOR,TORONTO RAPTORS,2018-19
2020_LAL,LOS ANGELES LAKERS,2019-20
2021_MIL,MILWAUKEE BUCKS,2020-21
2022_GSW,GOLDEN STATE WARRIORS,2021-22
2023_BOS,BOSTON CELTICS,2022-23
2024_DEN,DENVER NUGGETS,2023-24
2025_OKC,OKLAHOMA CITY THUNDER,2024-25


----------------------------------------------------------------------------------------------------------------------

# Será feito os datasets das métricas da liga em cada temporada

----------------------------------------------------------------------------------------------------------------------

## Para fazer de cada season:

In [62]:
get_all_teams_df()

,TEAM_NAME,TEAM_ID
0,ATLANTA HAWKS,1.610613e+09
1,BOSTON CELTICS,1.610613e+09
2,BROOKLYN NETS,1.610613e+09
3,CHARLOTTE HORNETS,1.610613e+09
4,CHICAGO BULLS,1.610613e+09
5,CLEVELAND CAVALIERS,1.610613e+09
6,DETROIT PISTONS,1.610613e+09
7,INDIANA PACERS,1.610613e+09
8,MIAMI HEAT,1.610613e+09
9,MILWAUKEE BUCKS,1.610613e+09


In [63]:
indexs = []
df_teams = get_all_teams_df()
for x in df_teams["TEAM_NAME"]:
    indexs.append(find_abb_by_name(x))

indexs_2016 = ['2016_' + item for item in indexs]
indexs_2017 = ['2017_' + item for item in indexs]
indexs_2018 = ['2018_' + item for item in indexs]
indexs_2019 = ['2019_' + item for item in indexs]
indexs_2020 = ['2020_' + item for item in indexs]
indexs_2021 = ['2021_' + item for item in indexs]
indexs_2022 = ['2022_' + item for item in indexs]
indexs_2023 = ['2023_' + item for item in indexs]
indexs_2024 = ['2024_' + item for item in indexs]
indexs_2025 = ['2025_' + item for item in indexs]

In [125]:
def generate_year_df(season):
    df_four_factors = get_four_factors(season)
    df_advanced_stats = get_advanced_stats(season)
    df_scoring_stats = get_advanced_stats(season)
    df_bench_pct = get_bench_point_percent(season)
    df_shot_locations = get_shot_locations_pct(season)
    df_usage = get_usage_stars(season)


    df_1 = pd.merge(df_four_factors, df_advanced_stats, on = "TEAM_ID")
    df_2 = pd.merge(df_scoring_stats, df_bench_pct, on="TEAM_ID")
    df_3 = pd.merge(df_shot_locations, df_usage, on= "TEAM_ID")
    df_4 = pd.merge(df_1, df_2, on="TEAM_ID")
    df_year = pd.merge(df_3, df_4, on="TEAM_ID")
    return df_year

# 2015-16

In [131]:
df_2016 = generate_year_df('2015-16')
df_2016

,TEAM_NAME,TEAM_ID,RIM_PCT,MID_PCT,SUM_USAGE,EFG_PCT,FTA_RATE,OREB_PCT,TM_TOV_PCT,OFF_RATING_x,DEF_RATING_x,NET_RATING_x,AST_PCT_x,PACE_x,OFF_RATING_y,DEF_RATING_y,NET_RATING_y,AST_PCT_y,PACE_y,BENCH_PTS_PCT
0,Atlanta Hawks,1610612737,0.350,0.197,0.6,0.516,0.237,0.234,0.152,104.6,100.8,3.9,0.663,97.63,104.6,100.8,3.9,0.663,97.63,0.336
1,Boston Celtics,1610612738,0.343,0.228,0.5,0.488,0.264,0.290,0.138,105.8,102.5,3.3,0.616,99.43,105.8,102.5,3.3,0.616,99.43,0.356
2,Brooklyn Nets,1610612751,0.316,0.265,0.5,0.492,0.246,0.279,0.153,102.4,109.7,-7.4,0.583,96.12,102.4,109.7,-7.4,0.583,96.12,0.331
3,Charlotte Hornets,1610612766,0.293,0.230,0.5,0.502,0.280,0.244,0.129,106.4,103.7,2.7,0.586,96.24,106.4,103.7,2.7,0.586,96.24,0.363
4,Chicago Bulls,1610612741,0.323,0.289,0.5,0.487,0.240,0.290,0.142,104.1,105.5,-1.4,0.591,96.60,104.1,105.5,-1.4,0.591,96.60,0.334
5,Cleveland Cavaliers,1610612739,0.298,0.213,0.6,0.524,0.259,0.294,0.143,110.2,103.9,6.3,0.587,93.82,110.2,103.9,6.3,0.587,93.82,0.270
6,Dallas Mavericks,1610612742,0.271,0.251,0.4,0.502,0.265,0.241,0.132,105.5,105.9,-0.4,0.592,95.34,105.5,105.9,-0.4,0.592,95.34,0.353
7,Denver Nuggets,1610612743,0.383,0.208,0.5,0.489,0.282,0.304,0.151,104.9,108.2,-3.3,0.601,96.34,104.9,108.2,-3.3,0.601,96.34,0.399
8,Detroit Pistons,1610612765,0.315,0.209,0.5,0.491,0.296,0.313,0.140,105.3,104.9,0.3,0.512,95.75,105.3,104.9,0.3,0.512,95.75,0.257
9,Golden State Warriors,1610612744,0.327,0.187,0.6,0.563,0.250,0.267,0.150,113.5,102.8,10.6,0.680,100.24,113.5,102.8,10.6,0.680,100.24,0.294


In [132]:
path = '../data/raw/df_2016.csv'
df_2016.to_csv(path, index=False)